In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline 

In [2]:
train = pd.read_csv("sendylogisticsproject/Train.csv")
test = pd.read_csv("sendylogisticsproject/Test.csv")
riders = pd.read_csv("sendylogisticsproject/Riders.csv")

In [3]:
train.shape, test.shape

((21201, 29), (7068, 25))

In [4]:
train.corr()

,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival
Platform Type,1.000000,0.012764,-0.058038,0.012770,-0.058005,0.012770,-0.058005,0.012770,-0.058005,0.012770,-0.058005,0.005528,0.225060,-0.055948,-0.002742,0.021327,0.006319,0.043924,-0.003827
Placement - Day of Month,0.012764,1.000000,0.026148,0.999998,0.026122,0.999998,0.026122,0.999998,0.026122,0.999998,0.026122,-0.007585,0.013097,-0.204143,-0.004929,-0.005453,0.007541,0.002077,-0.014710
Placement - Weekday (Mo = 1),-0.058038,0.026148,1.000000,0.026132,0.999952,0.026132,0.999952,0.026132,0.999952,0.026132,0.999952,0.003582,0.031064,-0.050592,-0.000918,0.011008,-0.004299,-0.000875,0.009693
Confirmation - Day of Month,0.012770,0.999998,0.026132,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,-0.007587,0.013099,-0.204143,-0.004923,-0.005448,0.007550,0.002073,-0.014701
Confirmation - Weekday (Mo = 1),-0.058005,0.026122,0.999952,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.003574,0.031074,-0.050592,-0.000883,0.011036,-0.004251,-0.000900,0.009744
Arrival at Pickup - Day of Month,0.012770,0.999998,0.026132,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,-0.007587,0.013099,-0.204143,-0.004923,-0.005448,0.007550,0.002073,-0.014701
Arrival at Pickup - Weekday (Mo = 1),-0.058005,0.026122,0.999952,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.003574,0.031074,-0.050592,-0.000883,0.011036,-0.004251,-0.000900,0.009744
Pickup - Day of Month,0.012770,0.999998,0.026132,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,-0.007587,0.013099,-0.204143,-0.004923,-0.005448,0.007550,0.002073,-0.014701
Pickup - Weekday (Mo = 1),-0.058005,0.026122,0.999952,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.003574,0.031074,-0.050592,-0.000883,0.011036,-0.004251,-0.000900,0.009744
Arrival at Destination - Day of Month,0.012770,0.999998,0.026132,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,1.000000,0.026123,-0.007587,0.013099,-0.204143,-0.004923,-0.005448,0.007550,0.002073,-0.014701


Calculate average speed in m/s and drop entries with outlying speeds

In [5]:
train['Average Speed']=(train['Distance (KM)']*1000)/train['Time from Pickup to Arrival']
train['Average Speed'].describe()
train=train[(train['Average Speed']>3) & (train['Average Speed']<21)]

In [6]:
train['Pickup - Time'] = pd.to_datetime(train['Pickup - Time'])
train['Arrival at Destination - Time'] = pd.to_datetime(train['Arrival at Destination - Time'])
train['Pickup_Hour'] = train['Pickup - Time'].dt.hour
train['Arrival_Hour'] = train['Arrival at Destination - Time'].dt.hour

### Rider data ###

In [ ]:
trainplusriders=train.copy()
trainplusriders=trainplusriders.set_index('Rider Id').join(riders.set_index('Rider Id'))
trainplusriders.reset_index()

In [ ]:
trainplusriders['Rider Median Speed']=trainplusriders.groupby(by='Rider Id')['Average Speed'].median()
trainplusriders.reset_index()

In [ ]:
trainplusriders[['No_Of_Orders', 'Age', 'Average_Rating', 'No_of_Ratings', 'Rider Median Speed', 'Time from Pickup to Arrival']].corr()

### Preprocessing

In [30]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.tree import ExtraTreeRegressor

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder 
toOHE=OneHotEncoder

from sklearn import metrics
from sklearn.model_selection import cross_validate

In [ ]:
train.shape

In [31]:
X=train.select_dtypes(np.number).fillna(0)

X=X.drop(columns=['Time from Pickup to Arrival', 'Precipitation in millimeters'])
X.loc[X['Temperature']==0, 'Temperature']=X['Temperature'].mean()
X=X[(X['Average Speed']>3) & (X['Average Speed']<21)]

print(X.columns, X.shape)
y=train['Time from Pickup to Arrival']

Index(['Platform Type', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Confirmation - Day of Month',
       'Confirmation - Weekday (Mo = 1)', 'Arrival at Pickup - Day of Month',
       'Arrival at Pickup - Weekday (Mo = 1)', 'Pickup - Day of Month',
       'Pickup - Weekday (Mo = 1)', 'Arrival at Destination - Day of Month',
       'Arrival at Destination - Weekday (Mo = 1)', 'Distance (KM)',
       'Temperature', 'Pickup Lat', 'Pickup Long', 'Destination Lat',
       'Destination Long', 'Average Speed', 'Pickup_Hour', 'Arrival_Hour'],
      dtype='object') (18219, 20)


In [32]:
num_data=X[['Pickup - Day of Month', 'Arrival at Destination - Day of Month',
            'Distance (KM)', 'Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long', 
            'Temperature', 'Arrival_Hour', 'Pickup_Hour', 'Average Speed']]
num_data.head(3)

,Pickup - Day of Month,Arrival at Destination - Day of Month,Distance (KM),Pickup Lat,Pickup Long,Destination Lat,Destination Long,Temperature,Arrival_Hour,Pickup_Hour,Average Speed
0,9,9,4,-1.317755,36.830370,-1.300406,36.829741,20.40000,10,10,5.369128
1,12,12,16,-1.351453,36.899315,-1.295004,36.814358,26.40000,12,11,8.028098
2,30,30,3,-1.308284,36.843419,-1.300921,36.828195,18.43964,13,12,6.593407


Days of the week columns converted to OHE vector

Platform type column not been used in below model

In [33]:
cat_data=X[['Pickup - Weekday (Mo = 1)', 'Arrival at Destination - Weekday (Mo = 1)']]
cat_data=cat_data.astype({'Pickup - Weekday (Mo = 1)': str, 'Arrival at Destination - Weekday (Mo = 1)': str})
cat_data.dtypes

Pickup - Weekday (Mo = 1)                    object
Arrival at Destination - Weekday (Mo = 1)    object
dtype: object

In [34]:
cat_data_ohe = pd.get_dummies(cat_data, dtype="int64")
cat_data_ohe.head(3)

,Pickup - Weekday (Mo = 1)_1,Pickup - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_3,Pickup - Weekday (Mo = 1)_4,Pickup - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_7,Arrival at Destination - Weekday (Mo = 1)_1,Arrival at Destination - Weekday (Mo = 1)_2,Arrival at Destination - Weekday (Mo = 1)_3,Arrival at Destination - Weekday (Mo = 1)_4,Arrival at Destination - Weekday (Mo = 1)_5,Arrival at Destination - Weekday (Mo = 1)_6,Arrival at Destination - Weekday (Mo = 1)_7
0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,1,0,0,0,0,0


### Model ###

Rider details yet to be used (like mean/median speed by rider id)

In [35]:
X=pd.concat([num_data, cat_data], axis=1)
num_data.shape, cat_data.shape

((18219, 11), (18219, 2))

In [36]:
lrmodel=LinearRegression()

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
scoring_functions = {"mae": "neg_mean_absolute_error", "r2": "r2"}
scores = cross_validate(lrmodel, X, y, scoring=scoring_functions, cv=5, return_train_score=True)
results_df = pd.DataFrame(scores)
results_df.mean().abs()

In [ ]:
lrmodel.fit(X=X_train, y=y_train)
predictions = lrmodel.predict(X_test)
X = X_test.reset_index().copy() 
X["target"] = y_test.tolist()
X["prediction"] = predictions
X.head()

In [ ]:
sns.scatterplot(x=X["Distance (KM)"], y=X["target"], label = 'Actual')
sns.scatterplot(x=X["Distance (KM)"], y=X["prediction"], label = 'Predicted')
sns.mpl.pyplot.ylabel("Time");

In [ ]:
sns.scatterplot(x=X['Pickup - Weekday (Mo = 1)'], y=X["target"], label = 'Actual')
sns.scatterplot(x=X['Pickup - Weekday (Mo = 1)'], y=X["prediction"], label = 'Predicted')
sns.mpl.pyplot.ylabel("Time");

In [ ]:
rmse = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print(f"RMSE ~ {int(rmse//60)} minutes")
mae = metrics.mean_absolute_error(y_test, predictions)
print(f"MAE ~ {mae//60} minutes)

### Ensemble

In [37]:
xrtmodel = BaggingRegressor(n_estimators=100, base_estimator=ExtraTreeRegressor())


In [38]:
abmodel = AdaBoostRegressor(n_estimators=100)

### Cross-validation

In [16]:
RESULTS = {}
scoring_functions = {"mae": "neg_mean_absolute_error", "r2": "r2"}
def evaluate_model(estimator):
    cv_results = cross_validate(estimator, X, y, scoring=scoring_functions, n_jobs=-1, cv=3,return_train_score=True)
    return pd.DataFrame(cv_results).abs().mean().to_dict()

In [57]:
RESULTS["linear_reg"] = evaluate_model(lrmodel)
RESULTS["bagging_random_tree"] = evaluate_model(xrtmodel)
RESULTS["adaboost"] = evaluate_model(abmodel)

In [58]:
pd.DataFrame.from_dict(RESULTS).T

,fit_time,score_time,test_mae,train_mae,test_r2,train_r2
linear_reg,0.091755,0.044279,201.428534,201.146589,0.864158,0.864721
bagging_random_tree,11.292049,3.875732,7.434528,3.451660,0.997998,0.999665
adaboost,12.662882,3.348856,245.243715,244.701416,0.886324,0.888305


### Tuning

In [54]:
search_parameters_space = {'n_estimators': range(1, 101), 'max_samples': range(1, 10), 'max_features': range(1, 14)}

In [55]:
random_search = RandomizedSearchCV(estimator=xrtmodel, param_distributions=search_parameters_space, scoring="neg_mean_absolute_error", n_jobs=-1, n_iter=50)

In [56]:
random_search.fit(X_train, y_train)
random_search.best_score_.abs(), random_search.best_params_

(-500.43537605904675,
 {'n_estimators': 33, 'max_samples': 9, 'max_features': 10})

In [71]:
abmodel.fit(X_train, y_train)

Index(['Pickup - Day of Month', 'Arrival at Destination - Day of Month',
       'Distance (KM)', 'Pickup Lat', 'Pickup Long', 'Destination Lat',
       'Destination Long', 'Temperature', 'Arrival_Hour', 'Pickup_Hour',
       'Average Speed', 'Pickup - Weekday (Mo = 1)',
       'Arrival at Destination - Weekday (Mo = 1)'],
      dtype='object')

In [73]:
abmodel.feature_importances_, X.columns

(array([1.76982316e-03, 3.69650409e-04, 5.43558486e-01, 4.29910083e-05,
        2.74024774e-04, 1.42834396e-02, 1.56953614e-03, 3.95936438e-04,
        1.29656134e-03, 3.71644945e-04, 4.35637984e-01, 0.00000000e+00,
        4.29921952e-04]),
 Index(['Pickup - Day of Month', 'Arrival at Destination - Day of Month',
        'Distance (KM)', 'Pickup Lat', 'Pickup Long', 'Destination Lat',
        'Destination Long', 'Temperature', 'Arrival_Hour', 'Pickup_Hour',
        'Average Speed', 'Pickup - Weekday (Mo = 1)',
        'Arrival at Destination - Weekday (Mo = 1)'],
       dtype='object'))

In [70]:
xrtmodel.fit(X_train, y_train)
predictions = xrtmodel.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print(f"RMSE ~ {rmse} seconds")
mae = metrics.mean_absolute_error(y_test, predictions)
print(f"MAE ~ {mae} seconds")

RMSE ~ 20.815594206194792 seconds
MAE ~ 5.146221185510426 seconds
